In [1]:
import random
import numpy as np
import pygame

pygame 2.1.0 (SDL 2.0.16, Python 3.9.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Board:
    def __init__(self, size=8):
        self.size = size
        self.board_matrix = np.zeros((self.size, self.size), dtype=np.int)
        self.valid_position = []
        self.last_position = None
        for i in range(size):
            for j in range(size):
                self.valid_position.append((i, j))

        self.player = 0

    def print_board(self):
        for i in range(self.size):
            for j in range(self.size):
                print(str(self.board_matrix[i][j]) + "   ", end="")
            print("\n")

    def set_board_matrix(self, mat):
        self.board_matrix = mat.copy()

    def is_valid_position(self, x, y):
        return 0 <= x < self.size and 0 <= y < self.size

    def has_valid_position(self):
        return len(self.valid_position) > 0

    def get_valid_position(self):
        length = len(self.valid_position)
        random_index = random.randint(0, length - 1)
        return self.valid_position[random_index]

    def remove_valid_position(self, position):
        self.valid_position.remove(position)

    def has_neighbor(self, position: tuple, radius):
        x = position[0]
        y = position[1]
        board = self.board_matrix
        start_x, end_x = (x - radius), (x + radius)
        start_y, end_y = (y - radius), (y + radius)

        for i in range(start_x, end_x + 1):
            for j in range(start_y, end_y + 1):
                if 0 <= i < self.size and 0 <= j < self.size:
                    if board[i][j] == 1 or board[i][j] == -1:
                        return True
        return False

    def has_neighbor_origin(self, x, y, radius):

        board = self.board_matrix
        start_x, end_x = (x - radius), (x + radius)
        start_y, end_y = (y - radius), (y + radius)

        for i in range(start_y, end_y + 1):
            for j in range(start_x, end_x + 1):
                if 0 <= i < self.size and 0 <= j < self.size:
                    if board[i][j] == 1 or board[i][j] == -1:
                        return True
        return False

    def get_has_neighbour_valid_position_by_random(self):
        # 生成子节点的时候选择有棋子的位置
        has_neighbour_position_list = []

        for position in self.valid_position:
            if self.has_neighbor(position, 1):
                has_neighbour_position_list.append(position)

        random_index = random.randint(0, len(has_neighbour_position_list) - 1)
        return has_neighbour_position_list[random_index]

    def move_in_position(self, position: tuple, player_flag: int):
        self.board_matrix[position[0]][position[1]] = player_flag

    def is_win(self, position: tuple):
        """
        检查当前position下子后是否有胜者

        :param position: (x,y)
        :return: true false
        """
        board = self.board_matrix
        x, y = position[0], position[1]
        player = board[x][y]

        # 横向
        for i in range(self.size - 4):
            count = 0
            for k in range(5):
                if i + k < self.size and board[x][i + k] == player:
                    count += 1
            if count >= 5:
                return True
        # 纵向
        for i in range(self.size - 4):
            count = 0
            for k in range(5):
                if i + k < self.size and board[i + k][y] == player:
                    count += 1
            if count >= 5:
                return True

        # -45度方向
        l1 = [board.diagonal(y - x)]
        # if (l1 == player).sum() >= 5:
        #     return True
        if len(l1) >= 5:
            for i in range(len(l1) - 4):
                count = 0
                for k in range(5):
                    if l1[i + k] == player:
                        count += 1
                if count >= 5:
                    return True

        # +45度方向
        l2 = [np.fliplr(board).diagonal(self.size - 1 - y - x)]
        # if (l2 == player).sum() >= 5:
        #     return True
        # print(l2)
        if len(l2) >= 5:
            for i in range(len(l2) - 4):
                count = 0
                for k in range(5):
                    if l2[i + k] == player:
                        count += 1
                if count >= 5:
                    return True
        return False

    # def cal_score(self, x, y):
    #     shape_score = {(0, 1, 1, 0, 0): 50,
    #                    (0, 0, 1, 1, 0): 50,
    #                    (1, 1, 0, 1, 0): 200,
    #                    (0, 0, 1, 1, 1): 500,
    #                    (1, 1, 1, 0, 0): 500,
    #                    (0, 1, 1, 1, 0): 5000,
    #                    (0, 1, 0, 1, 1, 0): 5000,
    #                    (0, 1, 1, 0, 1, 0): 5000,
    #                    (1, 1, 1, 0, 1): 5000,
    #                    (1, 1, 0, 1, 1): 5000,
    #                    (1, 0, 1, 1, 1): 5000,
    #                    (1, 1, 1, 1, 0): 5000,
    #                    (0, 1, 1, 1, 1): 5000,
    #                    (0, 1, 1, 1, 1, 0): 50000,
    #                    (1, 1, 1, 1, 1): 999999}


class BoardViewController:
    def __init__(self, board: Board) -> None:
        self.board = board

    def show_view(self):
        self.screen = pygame.display.set_mode(
            (50 * self.board.size, 50 * self.board.size)
        )
        pygame.display.set_caption("Interface of Five-in-a-Row")
        self.draw_view()

    def draw_view(self):
        self.screen.fill((230, 185, 70))
        for x in range(self.board.size):
            pygame.draw.line(
                self.screen,
                [0, 0, 0],
                [25 + 50 * x, 25],
                [25 + 50 * x, self.board.size * 50 - 25],
                1,
            )
            pygame.draw.line(
                self.screen,
                [0, 0, 0],
                [25, 25 + 50 * x],
                [self.board.size * 50 - 25, 25 + 50 * x],
                1,
            )
        pygame.display.update()

    def update_view(self):
        mat = self.board.board_matrix
        indices = np.where(mat != 0)
        for (row, col) in list(zip(indices[0], indices[1])):
            if mat[row][col] == 1:
                pygame.draw.circle(
                    self.screen, [0, 0, 0], [25 + 50 * col, 25 + 50 * row], 15, 15
                )
            elif mat[row][col] == -1:
                pygame.draw.circle(
                    self.screen, [255, 255, 255], [25 + 50 * col, 25 + 50 * row], 15, 15
                )
        pygame.display.update()

    def move_in_position(self, player_flag, position):
        is_end, is_win, action_done = False, False, False
        if self.board.board_matrix[position[0]][position[1]] == 0:
            self.board.move_in_position(position=position, player_flag=player_flag)
            self.board.remove_valid_position(position)
            self.board.last_position = position
            action_done = True
        if action_done:
            is_win = self.board.is_win(position=position)
            is_end = not self.board.has_valid_position()
            if is_win:
                print("winer is:", player_flag)
            self.update_view()
        return action_done, (is_win | is_end)

    def reset(self):
        self.board.reset()
        self.draw_view()


In [3]:
AI_SEARCH_DEPTH = 4
AI_LIMITED_MOVE_NUM = 20
from enum import IntEnum
from random import randint
import time

class CHESS_TYPE(IntEnum):
    NONE = 0,
    SLEEP_TWO = 1,
    LIVE_TWO = 2,
    SLEEP_THREE = 3
    LIVE_THREE = 4,
    CHONG_FOUR = 5,
    LIVE_FOUR = 6,
    LIVE_FIVE = 7,


CHESS_TYPE_NUM = 8

FIVE = CHESS_TYPE.LIVE_FIVE.value
FOUR, THREE, TWO = CHESS_TYPE.LIVE_FOUR.value, CHESS_TYPE.LIVE_THREE.value, CHESS_TYPE.LIVE_TWO.value
SFOUR, STHREE, STWO = CHESS_TYPE.CHONG_FOUR.value, CHESS_TYPE.SLEEP_THREE.value, CHESS_TYPE.SLEEP_TWO.value

SCORE_MAX = 0x7fffffff
SCORE_MIN = -1 * SCORE_MAX
SCORE_FIVE, SCORE_FOUR, SCORE_SFOUR = 100000, 10000, 1000
SCORE_THREE, SCORE_STHREE, SCORE_TWO, SCORE_STWO = 100, 10, 8, 2


class Evaluation():
    def __init__(self, chess_len):
        self.len = chess_len
        # [horizon, vertical, left diagonal, right diagonal]
        self.record = [[[0, 0, 0, 0] for x in range(chess_len)] for y in range(chess_len)]
        self.count = [[0 for x in range(CHESS_TYPE_NUM)] for i in range(2)]
    def genmove(self, board_c:Board, turn):
        board = board_c.board_matrix
        fives = []
        mfours, ofours = [], []
        msfours, osfours = [], []
        if turn == 1:
            mine = 1
            opponent = -1
        else:
            mine = -1
            opponent = 1

        moves = []
        radius = 1

        for y in range(self.len):
            for x in range(self.len):

                if board[y][x] == 0 and board_c.has_neighbor_origin(x, y, radius):
                    mscore, oscore = self.evaluatePointScore(board, x, y, mine, opponent)
                    point = (max(mscore, oscore), x, y)

                    if mscore >= SCORE_FIVE or oscore >= SCORE_FIVE:
                        fives.append(point)
                    elif mscore >= SCORE_FOUR:
                        mfours.append(point)
                    elif oscore >= SCORE_FOUR:
                        ofours.append(point)
                    elif mscore >= SCORE_SFOUR:
                        msfours.append(point)
                    elif oscore >= SCORE_SFOUR:
                        osfours.append(point)

                    moves.append(point)

        if len(fives) > 0: return fives

        if len(mfours) > 0: return mfours

        if len(ofours) > 0:
            if len(msfours) == 0:
                return ofours
            else:
                return ofours + msfours

        moves.sort(reverse=True)


        moves = moves[:AI_LIMITED_MOVE_NUM]
        return moves[0]

    def evaluatePointScore(self, board, x, y, mine, opponent):
        dir_offset = [(1, 0), (0, 1), (1, 1), (1, -1)]  # direction from left to right
        for i in range(len(self.count)):
            for j in range(len(self.count[0])):
                self.count[i][j] = 0

        board[y][x] = mine
        self.evaluatePoint(board, x, y, mine, opponent, self.count[mine - 1])
        mine_count = self.count[mine - 1]
        board[y][x] = opponent
        self.evaluatePoint(board, x, y, opponent, mine, self.count[opponent - 1])
        opponent_count = self.count[opponent - 1]
        board[y][x] = 0

        mscore = self.getPointScore(mine_count)
        oscore = self.getPointScore(opponent_count)

        return (mscore, oscore)


    def evaluatePoint(self, board, x, y, mine, opponent, count=None):
        dir_offset = [(1, 0), (0, 1), (1, 1), (1, -1)]  # direction from left to right
        ignore_record = True
        if count is None:
            count = self.count[mine - 1]
            ignore_record = False
        for i in range(4):
            if self.record[y][x][i] == 0 or ignore_record:
                self.analysisLine(board, x, y, i, dir_offset[i], mine, opponent, count)


    def getPointScore(self, count):
        score = 0
        if count[FIVE] > 0:
            return SCORE_FIVE

        if count[FOUR] > 0:
            return SCORE_FOUR

        if count[SFOUR] > 1:
            score += count[SFOUR] * SCORE_SFOUR
        elif count[SFOUR] > 0 and count[THREE] > 0:
            score += count[SFOUR] * SCORE_SFOUR
        elif count[SFOUR] > 0:
            score += SCORE_THREE

        if count[THREE] > 1:
            score += 5 * SCORE_THREE
        elif count[THREE] > 0:
            score += SCORE_THREE

        if count[STHREE] > 0:
            score += count[STHREE] * SCORE_STHREE
        if count[TWO] > 0:
            score += count[TWO] * SCORE_TWO
        if count[STWO] > 0:
            score += count[STWO] * SCORE_STWO

        return score

    def getLine(self, board, x, y, dir_offset, mine, opponent):
        line = [0 for i in range(9)]

        tmp_x = x + (-5 * dir_offset[0])
        tmp_y = y + (-5 * dir_offset[1])
        for i in range(9):
            tmp_x += dir_offset[0]
            tmp_y += dir_offset[1]
            if (tmp_x < 0 or tmp_x >= self.len or
                    tmp_y < 0 or tmp_y >= self.len):
                line[i] = opponent  # set out of range as opponent chess
            else:
                line[i] = board[tmp_y][tmp_x]

        return line

    def analysisLine(self, board, x, y, dir_index, dir, mine, opponent, count):
        # record line range[left, right] as analysized
        def setRecord(self, x, y, left, right, dir_index, dir_offset):
            tmp_x = x + (-5 + left) * dir_offset[0]
            tmp_y = y + (-5 + left) * dir_offset[1]
            for i in range(left, right + 1):
                tmp_x += dir_offset[0]
                tmp_y += dir_offset[1]
                self.record[tmp_y][tmp_x][dir_index] = 1

        empty = 0
        left_idx, right_idx = 4, 4

        line = self.getLine(board, x, y, dir, mine, opponent)

        while right_idx < 8:
            if line[right_idx + 1] != mine:
                break
            right_idx += 1
        while left_idx > 0:
            if line[left_idx - 1] != mine:
                break
            left_idx -= 1

        left_range, right_range = left_idx, right_idx
        while right_range < 8:
            if line[right_range + 1] == opponent:
                break
            right_range += 1
        while left_range > 0:
            if line[left_range - 1] == opponent:
                break
            left_range -= 1

        chess_range = right_range - left_range + 1
        if chess_range < 5:
            setRecord(self, x, y, left_range, right_range, dir_index, dir)
            return CHESS_TYPE.NONE

        setRecord(self, x, y, left_idx, right_idx, dir_index, dir)

        m_range = right_idx - left_idx + 1

        # M:mine chess, P:opponent chess or out of range, X: empty
        if m_range >= 5:
            count[FIVE] += 1

        # Live Four : XMMMMX
        # Chong Four : XMMMMP, PMMMMX
        if m_range == 4:
            left_empty = right_empty = False
            if line[left_idx - 1] == empty:
                left_empty = True
            if line[right_idx + 1] == empty:
                right_empty = True
            if left_empty and right_empty:
                count[FOUR] += 1
            elif left_empty or right_empty:
                count[SFOUR] += 1

        # Chong Four : MXMMM, MMMXM, the two types can both exist
        # Live Three : XMMMXX, XXMMMX
        # Sleep Three : PMMMX, XMMMP, PXMMMXP
        if m_range == 3:
            left_empty = right_empty = False
            left_four = right_four = False
            if line[left_idx - 1] == empty:
                if line[left_idx - 2] == mine:  # MXMMM
                    setRecord(self, x, y, left_idx - 2, left_idx - 1, dir_index, dir)
                    count[SFOUR] += 1
                    left_four = True
                left_empty = True

            if line[right_idx + 1] == empty:
                if line[right_idx + 2] == mine:  # MMMXM
                    setRecord(self, x, y, right_idx + 1, right_idx + 2, dir_index, dir)
                    count[SFOUR] += 1
                    right_four = True
                right_empty = True

            if left_four or right_four:
                pass
            elif left_empty and right_empty:
                if chess_range > 5:  # XMMMXX, XXMMMX
                    count[THREE] += 1
                else:  # PXMMMXP
                    count[STHREE] += 1
            elif left_empty or right_empty:  # PMMMX, XMMMP
                count[STHREE] += 1

        # Chong Four: MMXMM, only check right direction
        # Live Three: XMXMMX, XMMXMX the two types can both exist
        # Sleep Three: PMXMMX, XMXMMP, PMMXMX, XMMXMP
        # Live Two: XMMX
        # Sleep Two: PMMX, XMMP
        if m_range == 2:
            left_empty = right_empty = False
            left_three = right_three = False
            if line[left_idx - 1] == empty:
                if line[left_idx - 2] == mine:
                    setRecord(self, x, y, left_idx - 2, left_idx - 1, dir_index, dir)
                    if line[left_idx - 3] == empty:
                        if line[right_idx + 1] == empty:  # XMXMMX
                            count[THREE] += 1
                        else:  # XMXMMP
                            count[STHREE] += 1
                        left_three = True
                    elif line[left_idx - 3] == opponent:  # PMXMMX
                        if line[right_idx + 1] == empty:
                            count[STHREE] += 1
                            left_three = True

                left_empty = True

            if line[right_idx + 1] == empty:
                if line[right_idx + 2] == mine:
                    if line[right_idx + 3] == mine:  # MMXMM
                        setRecord(self, x, y, right_idx + 1, right_idx + 2, dir_index, dir)
                        count[SFOUR] += 1
                        right_three = True
                    elif line[right_idx + 3] == empty:
                        # setRecord(self, x, y, right_idx+1, right_idx+2, dir_index, dir)
                        if left_empty:  # XMMXMX
                            count[THREE] += 1
                        else:  # PMMXMX
                            count[STHREE] += 1
                        right_three = True
                    elif left_empty:  # XMMXMP
                        count[STHREE] += 1
                        right_three = True

                right_empty = True

            if left_three or right_three:
                pass
            elif left_empty and right_empty:  # XMMX
                count[TWO] += 1
            elif left_empty or right_empty:  # PMMX, XMMP
                count[STWO] += 1

        # Live Two: XMXMX, XMXXMX only check right direction
        # Sleep Two: PMXMX, XMXMP
        if m_range == 1:
            left_empty = right_empty = False
            if line[left_idx - 1] == empty:
                if line[left_idx - 2] == mine:
                    if line[left_idx - 3] == empty:
                        if line[right_idx + 1] == opponent:  # XMXMP
                            count[STWO] += 1
                left_empty = True

            if line[right_idx + 1] == empty:
                if line[right_idx + 2] == mine:
                    if line[right_idx + 3] == empty:
                        if left_empty:  # XMXMX
                            # setRecord(self, x, y, left_idx, right_idx+2, dir_index, dir)
                            count[TWO] += 1
                        else:  # PMXMX
                            count[STWO] += 1
                elif line[right_idx + 2] == empty:
                    if line[right_idx + 3] == mine and line[right_idx + 4] == empty:  # XMXXMX
                        count[TWO] += 1

        return CHESS_TYPE.NONE

In [4]:
import copy
import time

import numpy as np


class Node:
    def __init__(self, parent, position, board: Board):
        self.parent = parent
        self.is_visited = False
        self.position = position
        self.board = board
        self.num_visited = 0

        self.num_win = 0
        self.num_lose = 0

        self.children = []

    def get_uct(self, factor=np.sqrt(2)):
        v = (
            (self.num_win - self.num_lose) / self.num_visited
            if self.num_visited > 0
            else 0
        )
        return v + factor * np.sqrt(
            np.log(self.parent.num_visited) / (1 + self.num_visited)
        )

    def __str__(self):
        return f"{self.parent}-{self.position}"


class MCTS:
    def __init__(self, time, computational_power, first_position_in_MCTS, board):
        self.root = Node(parent=None, position=first_position_in_MCTS, board=board)
        self.fully_size = 8
        self.time = time
        self.computational_power = computational_power
        self.board_eval = Evaluation(board.size)

    def monte_carlo_tree_search(self):
        iter = 0
        self.plays_rave = {}  # key:move, value:visited times
        self.wins_rave = {}  # key:move, value:{player: win times}
        self.confident = 1.96
        self.equivalence = 1000
        while self.resources_left(time.time(), iter):
            self.visited_states = set()
            leaf = self.traverse_alpha(self.root)  # leaf = unvisited node
            simulation_result = self.rollout(leaf)
            self.backpropagate(leaf, simulation_result)
            iter += 1

        print(iter)
        print(time.time() - self.time)

        res = self.best_child_by_prob(self.root)
        print("(%d,%d)" % (res.position[0], res.position[1]))

        return res

    # For the traverse function, to avoid using up too much time or resources, you may start considering only
    # a subset of children (e.g 5 children). Increase this number or by choosing this subset smartly later.
    def traverse(self, node):
        while self.fully_expanded(node):
            node = self.best_uct_rave(node)

        if not self.non_terminal(node):

            return node

        else:

            return self.pick_univisted(node)
            # in case no children are present / node is terminal

    def traverse_alpha(self, node):

        while self.fully_expanded(node):
            node = self.best_ucb(node)
        if not self.non_terminal(node):
            return node

        else:

            return self.pick_univisted_has_neighbour(node)
            # in case no children are present / node is terminal

    def rollout(self, node):
        while self.non_terminal(node):
            node = self.rollout_policy(node)
        return self.result(node)

    def rollout_policy(self, node):

        return self.pick_has_neighbour_simulation(node)

    def backpropagate(self, node, result):

        self.update_stats(node, result)
        if node.parent is not None:
            self.backpropagate(node.parent, -result)

    def is_root(self, node):

        return node.parent is None

    def best_child(self, node):
        # pick child with highest number of visits
        visit_num_of_children = np.array(
            list([child.num_visited for child in node.children])
        )
        best_index = np.argmax(visit_num_of_children)

        return node.children[best_index]

    def best_child_by_prob(self, node):
        # pick child with highest win percent node

        win_percent_of_children = []
        for child in node.children:
            if child.num_visited > 0:
                win_percent_of_children.append(child.num_win / child.num_visited)
        best_index = np.argmax(win_percent_of_children)

        return node.children[best_index]

    def update_stats(self, node: Node, result):
        if result == 1:
            node.num_win += 1
        elif result == -1:
            node.num_lose += 1

        node.num_visited += 1
        return

    def fully_expanded(self, node: Node):

        return len(node.children) >= self.fully_size

    def resources_left(self, curr_time, iteration):
        if curr_time - self.time < 4.5 and iteration < self.computational_power:
            return True
        return False

    def non_terminal(self, node: Node):
        node_position = node.position
        if node.board.is_win(node_position) or node.board.has_valid_position() == False:
            # 出现胜者或者没有可下位置则代表棋局结束
            return False
        return True

    def pick_univisted(self, node):  # 生成子状态
        # node.board 对应旧状态
        new_board = copy.deepcopy(node.board)  # 得到旧状态的copy

        valid_position = new_board.get_valid_position()
        new_board.player *= -1  # 与父状态（node)交换下子方
        new_board.move_in_position(valid_position, new_board.player)
        new_board.remove_valid_position(valid_position)
        child = Node(parent=node, position=valid_position, board=new_board)
        node.children.append(child)
        return child

    def pick_univisted_has_neighbour(self, node):  # 生成子状态
        # node.board 对应旧状态
        plays_rave = self.plays_rave
        wins_rave = self.wins_rave
        new_board = copy.deepcopy(node.board)  # 得到旧状态的copy
        new_board.player *= -1  # 与父状态（node)交换下子方
        gen_move = self.board_eval.genmove(new_board, new_board.player)
        if isinstance(gen_move, list):
            gen_move = gen_move[0]
        pos = [gen_move[2], gen_move[1]]
        pos = tuple(pos)
        # valid_position_has_neighbour = new_board.get_has_neighbour_valid_position_by_random()

        new_board.remove_valid_position(pos)

        new_board.move_in_position(pos, new_board.player)
        child = Node(parent=node, position=pos, board=new_board)
        node.children.append(child)
        move = child.position
        player = child.board.player
        if move not in plays_rave:
            plays_rave[move] = 0
        if move in wins_rave:
            wins_rave[move][player] = 0
        else:
            wins_rave[move] = {player: 0}

        self.visited_states.add((player, move))
        return child

    def pick_random(self, node: Node):  # 随机下子并生成状态
        new_board = copy.deepcopy(node.board)

        valid_position = new_board.get_valid_position()
        new_board.remove_valid_position(valid_position)
        new_board.player *= -1  # 与父状态（node)交换下子方
        new_board.move_in_position(valid_position, new_board.player)
        child = Node(parent=node, position=valid_position, board=new_board)
        node.children.append(child)
        return child

    def pick_has_neighbour_simulation(self, node: Node):  # 随机下子并生成状态
        new_board = copy.deepcopy(node.board)

        valid_position = new_board.get_has_neighbour_valid_position_by_random()
        new_board.remove_valid_position(valid_position)
        new_board.player *= -1  # 与父状态（node)交换下子方
        new_board.move_in_position(valid_position, new_board.player)
        child = Node(parent=node, position=valid_position, board=new_board)
        node.children.append(child)
        return child

    def best_ucb(self, node: Node):
        uct_of_children = np.array(list([child.get_uct() for child in node.children]))
        best_index = np.argmax(uct_of_children)
        return node.children[best_index]

    def result(self, node):
        plays_rave = self.plays_rave
        wins_rave = self.wins_rave
        board = node.board
        node_position = node.position
        if board.is_win(node_position):
            winner = board.player
            for player, move in self.visited_states:
                if move in plays_rave:
                    plays_rave[move] += 1  # no matter which player
                    if winner in wins_rave[move]:
                        wins_rave[move][winner] += 1  # each move and every player
            return 1

        elif not board.has_valid_position():
            return 1 / 2

    def best_uct_rave(self, node: Node):
        uct_rave_of_children = np.array(
            list([self.get_uct_rave(child) for child in node.children])
        )

        best_index = np.argmax(uct_rave_of_children)
        return node.children[best_index]

    def get_uct_rave(self, node: Node):
        plays_rave = self.plays_rave
        wins_rave = self.wins_rave
        move = node.position
        res = (
            (1 - np.sqrt(self.equivalence / (3 * plays_rave[move] + self.equivalence)))
            * (node.num_win / node.num_visited)
            + np.sqrt(self.equivalence / (3 * plays_rave[move] + self.equivalence))
            * (wins_rave[move][node.board.player] / plays_rave[move])
            + np.sqrt(self.confident * np.log(plays_rave[move]) / node.num_visited)
        )

        return res


In [5]:
import concurrent.futures
import random
import time

import numpy as np
import pygame

class Player:
    def __init__(self, mat_flag=0) -> None:
        self.mat_flag = mat_flag

    def set_board_controller(self, board_controller):
        self.board_controller = board_controller

    def do_action(self, event=None):
        # stub
        # return is_win, action_done
        raise NotImplementedError


class HumanPlayer(Player):
    def do_action(self, event=None):
        """
        This function detects the mouse click on the game window. Update the state matrix of the game.
        input:
            event:pygame event, which are either quit or mouse click)
            mat: 2D matrix represents the state of the game
        output:
            mat: updated matrix
        """
        action_done, is_end = False, False
        if event.type == pygame.MOUSEBUTTONDOWN:
            (x, y) = event.pos
            # row = round((y - 40) / 40)
            # col = round((x - 40) / 40)
            row = round(abs(y - 25) / 50)
            col = round(abs(x - 25) / 50)
            action_done, is_end, = self.board_controller.move_in_position(
                position=(row, col), player_flag=self.mat_flag
            )
        return action_done, is_end


class AIPlayer(Player):
    def __init__(self, mat_flag=0, computational_power=50000) -> None:
        super().__init__(mat_flag=mat_flag)
        self.computational_power = computational_power

    @classmethod
    def get_ai_solution(cls, board, computational_power=50000):
        if np.count_nonzero(board.board_matrix == 0) == 0:
            return

        if np.all((board.board_matrix == 0)):
            x, y = board.board_matrix.shape
            return int(x >> 1), int(y >> 1)

        start_time = time.time()
        first_position_in_MCTS = board.last_position
        mcts1 = MCTS(
            start_time,
            computational_power,
            first_position_in_MCTS,
            board,
        )
        pos = mcts1.monte_carlo_tree_search().position
        return pos

    def do_action(self, event=None):
        """
        This is the core of the game. Write your code to give the computer the intelligence to play a Five-in-a-Row game
        with a human
        input:
            2D matrix representing the state of the game.
        output:
            2D matrix representing the updated state of the game.
        """
        board = self.board_controller.board

        pos = AIPlayer.get_ai_solution(board, self.computational_power)

        # ai suggestion
#         with concurrent.futures.ProcessPoolExecutor() as executor:
#             future = executor.submit(
#                 AIPlayer.get_ai_solution, board, self.computational_power
#             )
#             pos = future.result()
        if pos:
            action_done, is_end = self.board_controller.move_in_position(
                position=pos, player_flag=self.mat_flag
            )
        else:
            return True, True
        return action_done, is_end


In [6]:
import tkinter
import tkinter.messagebox

import pygame

# interface function for competition
def update_by_pc(mat, mat_flag=1):
    """
    This is the core of the game. Write your code to give the computer the intelligence to play a Five-in-a-Row game
    with a human
    input:
        2D matrix representing the state of the game.
    output:
        2D matrix representing the updated state of the game.
    """
    board = Board(size=max(mat.shape))
    board.set_board_matrix(mat)
    pos = AIPlayer.get_ai_solution(board)
    if pos:
        mat[pos[0], pos[1]] = mat_flag
    return mat


class Game:
    PLAYER_BLACK = 1
    PLAYER_WHITE = -1

    def __init__(self, board_size=15):
        self.dialog = tkinter.Tk()
        self.dialog.withdraw()

        pygame.init()
        pygame.key.set_repeat(10, 15)

        # init board view and controller
        self.board = Board(board_size)

        # init view
        self.board_controller = BoardViewController(self.board)

        # init players
        self.current_player = HumanPlayer(mat_flag=1)
        self.current_player.set_board_controller(self.board_controller)
        self.board.player = self.current_player.mat_flag

        # init ai player
        self.ai_player_1 = AIPlayer(mat_flag=1, computational_power=1000)
        self.ai_player_1.set_board_controller(self.board_controller)

        self.ai_player = AIPlayer(mat_flag=-1, computational_power=1000)
        self.ai_player.set_board_controller(self.board_controller)

        # init game parameters
        self.turn = 0
        self.done = False

    def get_player_name(self, turn):
        return "black" if turn % 2 else "white"

    def start_looper(self):
        while not self.done:
            # event = pygame.event.wait()
            # if event.type == pygame.QUIT:
            #     self.done = True
            #     exit()
            # if event.type in [pygame.MOUSEBUTTONDOWN] and not (self.turn % 2):
            ###it is player 1 turn
            # action_done, self.done = self.current_player.do_action(event)
            # if not action_done:
            #     continue
            _, self.done = self.ai_player_1.do_action()
            self.turn += 1
            ###it is player 1 turn end

            ### it is ai player turn
            _, self.done = self.ai_player.do_action()
            self.turn += 1
            ###it is player 2 turn end

            if self.done:
                self.show_endgame_dialog()
            pygame.display.update()

    def reset(self):
        self.board_controller.reset()
        self.turn = 0
        self.done = False

    def start(self):
        self.board_controller.show_view()
        self.start_looper()

    def show_endgame_dialog(self):
        retry = tkinter.messagebox.askretrycancel(
            "Game Over",
            f"Do you want to try again?",
        )
        if retry:
            self.reset()
        else:
            exit()
        self.dialog.lift()


In [ ]:
game = Game()
game.start()

/var/folders/6d/5m41q68579l8w1506v9dcgjr0000gn/T/ipykernel_63856/2221619904.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.board_matrix = np.zeros((self.size, self.size), dtype=np.int)


41
4.5636279582977295
(8,8)
38
4.540897846221924
(7,8)
44
4.545428991317749
(7,9)
38
4.52827000617981
(6,10)
35
4.583350896835327
(6,8)
38
4.547911167144775
(5,7)
39
4.568300008773804
(8,7)
48
4.655152082443237
(8,9)
37
4.632446050643921
(6,7)
40
4.581336975097656
(6,6)
43
4.594357013702393
(4,8)
41
4.616174221038818
(7,6)
54
4.6003148555755615
(8,6)
40
4.574005126953125
(7,5)
34
4.613270998001099
(7,4)
36
4.546030044555664
(8,5)
35
4.573519229888916
(6,5)
38
4.523140907287598
(5,6)
50
4.5136168003082275
(5,8)
43
4.508238077163696
(4,6)
47
4.512105941772461
(3,6)
45
4.536664009094238
(8,4)
43
4.553618907928467
(9,3)
45
4.571699857711792
(4,7)
47
4.520186185836792
(3,8)
1000
0.1638500690460205
(2,8)
42
4.5760369300842285
(3,7)
60
4.535928010940552
(3,5)
38
4.5296289920806885
(2,6)
47
4.5232930183410645
(1,5)
42
4.5147809982299805
(4,5)
48
4.523568868637085
(2,4)
49
4.506242752075195
(1,3)
46
4.500717639923096
(3,9)
51
4.563414812088013
(2,7)
40
4.598924875259399
(1,8)
44
4.5830001831054